<a href="https://colab.research.google.com/github/Aliaksandr-Borsuk/Recommender_Systems_project/blob/main/notebooks/02_knn_like.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка


**Цель:**
- построить и сравнить простые KNN-модели, подобрать параметры, оценить на валидации и тесте   
**Данные:**
- используем train, test  из предыдущего ноутбука
- бьём train на  v_train, val для подбора гиперпараметров

## 01. Клонируем репозиторий.Подключаем GoogleDrive.

In [9]:
!rm -rf /content/Recommender_Systems_project
!git clone https://github.com/Aliaksandr-Borsuk/Recommender_Systems_project

Cloning into 'Recommender_Systems_project'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 85 (delta 34), reused 41 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 489.75 KiB | 4.41 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [4]:
# подключаем диск
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=10790422 sha256=1fa55ce930395395805ec3cf6b2c64cb6dc9884a09e2d2ccfdf9a13aa2367a53
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.2 MB/s eta 0:00:00


## 02. Импорты

In [11]:
import sys
sys.path.append("/content/Recommender_Systems_project/src")

import optuna
from optuna.samplers import TPESampler

import time
import pickle
import implicit
import warnings
warnings.filterwarnings("ignore",  module="implicit")
from scipy.sparse import save_npz, load_npz, csr_matrix
from implicit.nearest_neighbours import CosineRecommender, BM25Recommender, TFIDFRecommender

from pprint import pprint
from pathlib import Path
import pandas as pd
import numpy as np
import random

# импортируем нужные функци
from recommender.data_io import train_test_reader # для чтения сохранённых из 001_data_and_eda_1m_proba
from recommender.splitters import df_time_split  # для time_split разбиения df
from recommender.preprocessing import prepare_knn_matrix  # для получения матрицы взаимодействий
from recommender.metrics import hitrate_at_k, coverage_at_k, precision_at_k,\
                     recall_at_k, ndcg_at_k, map_at_k, model_evaluation
from recommender.results_logger import save_experiment_results # для сохранения результатов


# from recommender.similarity import build_similarity_matrix # выдаёт матрицу схожести между объектами (item или user)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

DATA =   Path("/content/drive/MyDrive/Colab Notebooks/data/")
RAW_DATA = DATA/"raw"
PROCESSED = DATA/"processed"
RESULTS_DIR = DATA / "results"
DATA.mkdir(exist_ok=True)

## 03. Грузим train, test, meta_данные.



In [8]:
train_tast_path = '/content/drive/MyDrive/Colab Notebooks/data/processed/251021_173655'

train, test, meta = train_test_reader(train_tast_path)
pprint(meta, width=80, compact=False)
print(f'\ntrain shape : {train.shape}')
print(f'test shape  : {test.shape}')
print( '\n', '*'*50, '\ntrain.head')
display(train.head(3))
print('\n', '*'*50, '\ntest.head')
display(test.head(3))

{'columns': ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres'],
 'created_at': '2025-10-21T17:37:00.607645',
 'min_test_interactions': 10,
 'min_train_interactions': 5,
 'n_items': 3662,
 'n_test_users': 836,
 'n_train_users': 5392,
 'test_shape': [94842, 6],
 'time_treshold': '2000-12-02T14:52:18',
 'train_shape': [800142, 6]}

train shape : (800142, 6)
test shape  : (94842, 6)

 ************************************************** 
train.head


,user_id,item_id,rating,timestamp,title,genres
0,635,1251,4,975768620,8 1/2 (1963),Drama
1,635,3948,4,975768294,Meet the Parents (2000),Comedy
2,635,1270,4,975768106,Back to the Future (1985),Comedy|Sci-Fi



 ************************************************** 
test.head


,user_id,item_id,rating,timestamp,title,genres
0,635,3789,5,975768788,"Pawnbroker, The (1965)",Drama
1,635,2987,5,979141847,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
2,635,2988,4,975769007,Melvin and Howard (1980),Drama


## 04. Бьем Train на v_train и val для подбора гиперпараметров
Используем опять вариант **B** - разбиение по времени(Time-based split).
 Все события до порога в v_train, события после в val. Это гарантирует, что ни одно событие из "будущего" не попадёт в v_train.

In [ ]:
# Бьем Train на v_train и val
# колонки для сохранения
columns_to_save = ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres']
user_id = 'user_id'
item_id = 'item_id'
time_column = 'timestamp'

# минимальное тёплое число оценок
min_n_reitings = 5
# min кол-во оценок у каждого юзера в train и в test
n, k = 15, 10
quantile = 0.85    # квантиль разбиения
# разбиение
v_train, val = df_time_split(train, time_column, columns_to_save,
                             user_id , item_id,
                             min_n_reitings, n, k , quantile)

users_intersection = set(val['user_id']) & set(v_train['user_id'])
print('*'*70)
print(f"\n{len(users_intersection)} юзера встречаются одновременно и в v_train и в val")

# залистим тестовых пользователей
val_users = sorted(users_intersection)

print('\ntrain ')
display(v_train.head(2))
print('\ntest ')
display(val.head(2))

Порог разбиения по времени 974814617
Размеры после разбиения: train 680122 test 120020

Осталось пользователей в train : 4748
Новый размер train: (680108, 6)
Осталось пользователей в test: 211
Новый размер test: (26076, 6)
**********************************************************************

211 юзера встречаются одновременно и в v_train и в val

train 


,user_id,item_id,rating,timestamp,title,genres
0,1272,720,5,974814383,Wallace & Gromit: The Best of Aardman Animatio...,Animation
1,1272,2993,2,974814617,Thunderball (1965),Action



test 


,user_id,item_id,rating,timestamp,title,genres
0,1272,2987,4,974822814,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
1,1272,2989,4,974814713,For Your Eyes Only (1981),Action


**Краткий вывод:**  
Подбирать гиперпараметры можно, но результаты будут с высокой дисперсией и ограниченной статистической надёжностью: валидационный пул - 211 пользователей  маловат для уверенных выводов по top‑10, особенно если пользователи неоднородны.  
  
---

Можно сохранять несколько лучших конфигураций.)

## 05. Подготовка(и сохранение) валидационной матрицы взаимодействий

In [ ]:
# матрица и словарики
v_train_matrix, user2index_v, item2index_v = prepare_knn_matrix(v_train, threshold=0.0)
index2user_v = {v: k for k, v in user2index_v.items()}
index2item_v = {v: k for k, v in item2index_v.items()}

# Сохранение
out_dir = PROCESSED/"artifacts"
out_dir.mkdir(exist_ok=True)

# Сохранение матрицы взаимодействий
save_npz(out_dir/"v_train_matrix.npz", v_train_matrix)

# Словари индексации
with open(out_dir/"user2index_v.pkl", "wb") as f:
    pickle.dump(user2index_v, f)

with open(out_dir/"item2index_v.pkl", "wb") as f:
    pickle.dump(item2index_v, f)

# Обратные словари
with open(out_dir/"index2user_v.pkl", "wb") as f:
    pickle.dump(index2user_v, f)

with open(out_dir/"index2item_v.pkl", "wb") as f:
    pickle.dump(index2item_v, f)

## 06. Пробная загрузка

In [ ]:
input_dir = PROCESSED/"artifacts"

# Загрузка матрицы взаимодействий
v_train_matrix = load_npz(input_dir / "v_train_matrix.npz")

# Загрузка словарей
with open(input_dir / "user2index_v.pkl", "rb") as f:
    user2index_v = pickle.load(f)

with open(input_dir / "item2index_v.pkl", "rb") as f:
    item2index_v = pickle.load(f)

with open(input_dir / "index2user_v.pkl", "rb") as f:
    index2user_v = pickle.load(f)

with open(input_dir / "index2item_v.pkl", "rb") as f:
    index2item_v = pickle.load(f)

v_train_matrix
assert isinstance(v_train_matrix, csr_matrix), "train_matrix должен быть csr_matrix"

# **потрогаем KNN рекомендации**
- и за itemsы и за usersы....)

## 07. item_KNN c implicit
**implicit** - это библиотека на Python для построения рекомендательных систем на основе implicit feedback (скрытых сигналов, например просмотров, кликов, лайков), с быстрыми реализациями популярных алгоритмов и поддержкой CPU/GPU.   

---

**Назначение:**   
 библиотека для коллаборативной фильтрации по данным **implicit feedback**, где нет явных рейтингов, а есть только факты взаимодействия (просмотр, покупка, лайк).

**Основные алгоритмы:**  

- **ALS (Alternating Least Squares)** - матричная факторизация для implicit feedback.

- **BPR (Bayesian Personalized Ranking)** - оптимизация ранжирования.

- **Logistic Matrix Factorization.** - моделирует вероятность взаимодействия пользователя с объектом через логистическую функцию, а не через простое скалярное произведение латентных векторов.

- **Item-Item Nearest Neighbour** (с косинусным расстоянием, TF-IDF, BM25).  

**Производительность:**

Использует Cython + OpenMP для многопоточной работы на CPU.  

Есть CUDA-ядра для GPU (если драйвер и CUDA совместимы).  

**Интеграции:**   
 поддерживает библиотеки для Approximate Nearest Neighbours (Annoy, NMSLIB, Faiss) для ускорения поиска рекомендаций.

**Применение:**  
 удобно для задач типа «порекомендуй фильм/товар/трек на основе истории взаимодействий».   

 ---
   
**Важные особенности**  
- Работает именно с implicit feedback, а не с явными рейтингами.

- Подходит для больших датасетов (MovieLens, e-commerce, music streaming).

- Простая интеграция: можно обучить модель и сразу получать рекомендации для пользователей или похожие объекты.

- Поддерживает сохранение и загрузку моделей.

### 07_1. подбор гиперпараметров Optuna





In [ ]:
# заменяем реальные ID на индексы из матрицы обучения модели
val_mapped = val.assign(
    user_id = val["user_id"].map(user2index_v),
    item_id = val["item_id"].map(item2index_v)
)
assert val_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
val_items = val_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items
all_items = set(v_train['item_id'].map(item2index_v).dropna().astype(int).unique())

In [ ]:
def recommend_with_model(model, train_matrix, val_users, topn=10) -> dict[int, list[int]]:
    """
    Генерация рекомендаций для всех пользователей.
    """
    # model.recommend возвращает два массива дл якаждого юзераitems и scores
    items, scores = model.recommend(val_users, train_matrix[val_users], N=topn)
    items = items.tolist()
    recs = {val_users[i]: items[i] for i in range(len(val_users))}
    return recs


In [ ]:
# подбор гиперпараметров Optuna
def make_objective_itemKNN (train_matrix, val_items, all_items,
                    # kind="item",
                               metric_fn=ndcg_at_k, k_eval=10):

    # Приведение к нужному типу для implisit
    user_items = train_matrix.tocsr(copy=True)

    user_items.indptr  = user_items.indptr.astype(np.int32, copy=False)
    user_items.indices = user_items.indices.astype(np.int32, copy=False)
    user_items.data    = user_items.data.astype(np.float32, copy=False)


    def objective(trial):
        # подбираем число соседей
        K = trial.suggest_int("K", 1, 1000, log=True)
        sim = trial.suggest_categorical("similarity", ["bm25",  "cosine", "tfidf" ])

        # создаём модель
        if sim == "cosine":
            model = implicit.nearest_neighbours.CosineRecommender(K=K)
        elif sim == "bm25":
            model = implicit.nearest_neighbours.BM25Recommender(K=K)
        elif sim == "tfidf":
            model = implicit.nearest_neighbours.TFIDFRecommender(K=K)

        model.fit(user_items, show_progress=False)

        # рекомендации
        rec_items = recommend_with_model(model, user_items, list(val_items.keys()), topn=k_eval)

        # метрика
        score = metric_fn(rec_items, val_items, k=k_eval)

        #print(f'\ntrial {kind}_KNN with: simylarity = {sim}  K = {K}, score = {score}\n')
        return score

    return objective

In [ ]:
n_trials=50
# замерим время
start = time.time()

sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)

# item-KNN
study_item = optuna.create_study(direction="maximize", sampler=sampler)
study_item.optimize(
    make_objective_itemKNN(v_train_matrix, val_items, all_items,
                      metric_fn=ndcg_at_k, k_eval=10),
    n_trials=n_trials
)

end = time.time()
study_item_time = end - start

print('#'*70)
print("\nЛучшие параметры item-KNN:", study_item.best_params)
print("Лучший score:", study_item.best_value)

print('\n')
print('*'*70)
print(f"Время выполнения: {study_item_time:.2f} секунд")

[I 2025-11-08 08:34:54,813] A new study created in memory with name: no-name-a777b6da-df21-480d-ba84-b5a6f8cb5a15
[I 2025-11-08 08:34:56,829] Trial 0 finished with value: 0.19877434759347073 and parameters: {'K': 9, 'similarity': 'bm25'}. Best is trial 0 with value: 0.19877434759347073.
[I 2025-11-08 08:34:58,401] Trial 1 finished with value: 0.2391795750993354 and parameters: {'K': 2, 'similarity': 'tfidf'}. Best is trial 1 with value: 0.2391795750993354.
[I 2025-11-08 08:35:00,039] Trial 2 finished with value: 0.3023988373616022 and parameters: {'K': 48, 'similarity': 'tfidf'}. Best is trial 2 with value: 0.3023988373616022.
[I 2025-11-08 08:35:01,127] Trial 3 finished with value: 0.2817891083501537 and parameters: {'K': 280, 'similarity': 'bm25'}. Best is trial 2 with value: 0.3023988373616022.
[I 2025-11-08 08:35:01,684] Trial 4 finished with value: 0.1923573888672695 and parameters: {'K': 5, 'similarity': 'bm25'}. Best is trial 2 with value: 0.3023988373616022.
[I 2025-11-08 08:35

######################################################################

Лучшие параметры item-KNN: {'K': 312, 'similarity': 'tfidf'}
Лучший score: 0.3333083448396949


**********************************************************************
Время выполнения: 59.32 секунд


In [ ]:
# optuna визуализация
print("OPTUNA for item_KNN")
optuna.visualization.plot_optimization_history(study_item).show()
optuna.visualization.plot_param_importances(study_item).show()
optuna.visualization.plot_parallel_coordinate(study_item).show()

OPTUNA for item_KNN


#### **Вывод:**
- Optuna нашёл хорошее K в средней части диапазона (несколько сотен)
 - из similarity tfidf и  cosine предпочтительнее для itemKNN.
- Optuna - молодец!

## 08. user_KNN c implicit  
- т.к. implisit не поддерживает UserKNN из коробки. То немножк попляшем с бубнами и слепим userKNN ручкаме.

In [ ]:
import numpy as np
import numbers
import scipy.sparse as sp
import warnings
from scipy.sparse import SparseEfficiencyWarning
from implicit.nearest_neighbours import CosineRecommender, BM25Recommender, TFIDFRecommender

class PET_UserKNN_with_Implicit:
    '''
    должен уметь :
    - считать similarity - 'cosine', 'bm25', 'tfidf'
    - для разного числа соседей K
    - выдавать рекоммендации как для одгног юзера , так и для пула
    должен поддерживать методы:
     .fit(train_matrix, show_progress=True) train_matrix - матрица users*items
     .recommend(val_users, train_matrix, N=topn)
    '''
    def __init__(self, K=50, similarity="cosine"):
        """
        K - число соседей
        similarity - 'cosine', 'bm25', 'tfidf'
        """
        self.K = K
        self.similarity = similarity
        self.model = None
        self.train_matrix = None

    def fit(self, train_matrix, show_progress=True):
        """
        Обучение user-KNN через implicit.
        train_matrix: csr_matrix (users × items), взаимродействие - (0/1)
        """
        self.train_matrix = train_matrix.tocsr(copy=True)
        user_users = train_matrix.T.tocsr(copy=True)  # теперь "items" = пользователи

        if self.similarity == "cosine":
            self.model = CosineRecommender(K=self.K)
        elif self.similarity == "bm25":
            self.model = BM25Recommender(K=self.K)
        elif self.similarity == "tfidf":
            self.model = TFIDFRecommender(K=self.K)
        else:
            raise ValueError("similarity должен быть 'cosine', 'bm25' или 'tfidf'")

        self.model.fit(user_users, show_progress=show_progress)

    def recommend_one(self, user_id, N=10):
        """
        Рекомендации для одного пользователя.
        """
        similar_users, sim_scores = self.model.similar_items(user_id, N=self.K)

        # агрегируем взаимодействия соседей с весами
        neigh_matrix = self.train_matrix[similar_users]   # соседи × items
        weighted = sim_scores @ neigh_matrix.toarray()    # вектор длиной items

        # убираем просмотренные
        seen = self.train_matrix[user_id].indices
        weighted[seen] = -np.inf

        # топ-N
        top_items = np.argpartition(-weighted, N)[:N]
        scores = weighted[top_items]
        order = np.argsort(-scores)
        return top_items[order], scores[order]

    def recommend_batch_matrix(self, user_ids, N=10):
        """
        Рекомендации user-KNN для пула пользователей.
        Возвращает два массива:
          items  – shape (len(user_ids), topn)
          scores – shape (len(user_ids), topn)
        """
        batch_size = len(user_ids)
        n_users, n_items = self.train_matrix.shape

        # 1. Получаем соседей и их веса сразу для всех user_ids
        similar_users, sim_scores = self.model.similar_items(user_ids, N=self.K)

        # 2. Строим sparse матрицу весов (batch × users)
        rows = np.repeat(np.arange(batch_size), self.K)
        cols = similar_users.ravel()
        data = sim_scores.ravel()
        S_batch = sp.csr_matrix((data, (rows, cols)), shape=(batch_size, n_users))

        # 3. Агрегируем взаимодействия (batch × items)
        W = S_batch @ self.train_matrix   # csr_matrix

        # 4. Маскируем просмотренные - полностью матрично
        seen_batch = self.train_matrix[user_ids]          # csr_matrix (batch × items)
        mask = seen_batch.astype(bool)         # True там, где были взаимодействия
        # зануляем предупреждение о временном уплотнении матрицы с шумодавом), ибо, ибо....
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", SparseEfficiencyWarning)
            W = W.multiply(seen_batch == 0)            # зануляем просмотренные

        # 5. Выбираем top-N построчно
        W_dense = W.toarray()                        # dense только для argpartition
        top_items = np.argpartition(-W_dense, N, axis=1)[:, :N]
        top_scores = np.take_along_axis(W_dense, top_items, axis=1)

        order = np.argsort(-top_scores, axis=1)
        items_sorted = np.take_along_axis(top_items, order, axis=1)
        scores_sorted = np.take_along_axis(top_scores, order, axis=1)

        return items_sorted, scores_sorted

    def recommend(self, users_id, train_matrix=None, N=10):
      if isinstance(users_id, numbers.Number):
          return self.recommend_one(users_id, N)
      elif isinstance(users_id, list):
          return self.recommend_batch_matrix(users_id, N)




### 08_1. подбор гиперпараметров Optuna

In [ ]:
# подбор гиперпараметров Optuna
def make_objective_userKNN (train_matrix, val_items, all_items,
                               metric_fn=ndcg_at_k, k_eval=10):
    # Приведение к нужному типу для implisit внутри userKNN

    def objective(trial):
        # подбираем число соседей
        K = trial.suggest_int("K", 1, 1000, log=True)
        sim = trial.suggest_categorical("similarity", ["bm25",  "cosine", "tfidf" ])

        # создаём модель
        if sim == "cosine":
            model = PET_UserKNN_with_Implicit(K=K, similarity="cosine")
        elif sim == "bm25":
            model = PET_UserKNN_with_Implicit(K=K, similarity="bm25")
        elif sim == "tfidf":
            model = PET_UserKNN_with_Implicit(K=K, similarity="tfidf")

        model.fit(train_matrix, show_progress=False)

        # рекомендации
        rec_items = recommend_with_model(model, train_matrix, list(val_items.keys()), topn=k_eval)

        # метрика
        score = metric_fn(rec_items, val_items, k=k_eval)

        return score

    return objective

In [ ]:
n_trials = n_trials

# замерим время
start = time.time()


sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)

# user-KNN
study_user = optuna.create_study(direction="maximize", sampler=sampler)
study_user.optimize(
    make_objective_userKNN(v_train_matrix, val_items, all_items,
                    metric_fn=ndcg_at_k, k_eval=10),
    n_trials=n_trials
)

end = time.time()
study_user_time = end - start

print('#'*70)
print("\nЛучшие параметры user-KNN:", study_user.best_params)
print("Лучший score:", study_user.best_value)

print('\n')
print('*'*70)
print(f"Время выполнения : {study_user_time:.2f} секунд")

[I 2025-11-08 08:35:55,367] A new study created in memory with name: no-name-8a282169-b83a-41c8-ad89-550bf2715a47
[I 2025-11-08 08:35:57,694] Trial 0 finished with value: 0.19893734162467355 and parameters: {'K': 9, 'similarity': 'bm25'}. Best is trial 0 with value: 0.19893734162467355.
[I 2025-11-08 08:35:59,188] Trial 1 finished with value: 0.12836170376330788 and parameters: {'K': 2, 'similarity': 'tfidf'}. Best is trial 0 with value: 0.19893734162467355.
[I 2025-11-08 08:36:01,168] Trial 2 finished with value: 0.27756518485957044 and parameters: {'K': 48, 'similarity': 'tfidf'}. Best is trial 2 with value: 0.27756518485957044.
[I 2025-11-08 08:36:03,433] Trial 3 finished with value: 0.3255863183556515 and parameters: {'K': 280, 'similarity': 'bm25'}. Best is trial 3 with value: 0.3255863183556515.
[I 2025-11-08 08:36:04,964] Trial 4 finished with value: 0.18839384722636573 and parameters: {'K': 5, 'similarity': 'bm25'}. Best is trial 3 with value: 0.3255863183556515.
[I 2025-11-08 

######################################################################

Лучшие параметры user-KNN: {'K': 991, 'similarity': 'bm25'}
Лучший score: 0.35921731644010224


**********************************************************************
Время выполнения : 113.18 секунд


In [ ]:
# Визуализируем
print("OPTUNA for user_KNN")
optuna.visualization.plot_optimization_history(study_user).show()
optuna.visualization.plot_param_importances(study_item).show()
optuna.visualization.plot_parallel_coordinate(study_user).show()

OPTUNA for user_KNN


#### **Вывод:**
- Optuna нашёл хорошее K в верхней части диапазона.
 - из similarity tfidf и  bm25 предпочтительнее для userKNN.
- Optuna - молодец!

## 09. Получение метрик на test

### Подготовка(и сохранение)  матрицы взаимодействий для получения рекомендаций на test.

In [ ]:
# матрица и словарики
train_matrix, user2index, item2index = prepare_knn_matrix(train, threshold=0.0)
index2user = {v: k for k, v in user2index.items()}
index2item = {v: k for k, v in item2index.items()}

# Сохранение
out_dir = PROCESSED/"artifacts"
out_dir.mkdir(exist_ok=True)

# Сохранение матрицы взаимодействий
save_npz(out_dir/"train_matrix.npz", train_matrix)

# Словари индексации
with open(out_dir/"user2index.pkl", "wb") as f:
    pickle.dump(user2index, f)

with open(out_dir/"item2index.pkl", "wb") as f:
    pickle.dump(item2index, f)

# Обратные словари
with open(out_dir/"index2user.pkl", "wb") as f:
    pickle.dump(index2user, f)

with open(out_dir/"index2item.pkl", "wb") as f:
    pickle.dump(index2item, f)

### Загрузка матрицы взаимодействий для получения рекомендаций на test..

In [ ]:
# загрузка
input_dir = PROCESSED/"artifacts"

# Загрузка матрицы взаимодействий
train_matrix = load_npz(input_dir / "train_matrix.npz")

# Загрузка словарей
with open(input_dir / "user2index.pkl", "rb") as f:
    user2index = pickle.load(f)

with open(input_dir / "item2index.pkl", "rb") as f:
    item2index = pickle.load(f)

with open(input_dir / "index2user.pkl", "rb") as f:
    index2user = pickle.load(f)

with open(input_dir / "index2item.pkl", "rb") as f:
    index2item = pickle.load(f)

assert isinstance(train_matrix, csr_matrix), "train_matrix должен быть csr_matrix"
train_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 800142 stored elements and shape (5392, 3662)>

In [ ]:
# заменяем реальные ID на индексы
test_mapped = test.assign(
    user_id = test["user_id"].map(user2index),
    item_id = test["item_id"].map(item2index)
)
assert test_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
test_items = test_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items
all_items = set(train['item_id'].map(item2index).dropna().astype(int).unique())

### лепим рекомендации

In [ ]:
# Вспомним лучшее:
print('Лучшие парамеры itemKNN :',study_item.best_params)
print('Лучшие парамеры userKNN :',study_user.best_params)

Лучшие парамеры itemKNN : {'K': 312, 'similarity': 'tfidf'}
Лучшие парамеры userKNN : {'K': 991, 'similarity': 'bm25'}


In [ ]:
# fit: item-based → train_matrix.T, user-based → train_matrix
user_items = train_matrix.tocsr(copy=True)

user_items.indptr  = user_items.indptr.astype(np.int32, copy=False)
user_items.indices = user_items.indices.astype(np.int32, copy=False)
user_items.data    = user_items.data.astype(np.float32, copy=False)


# --- userKNN (лучшие параметры) ---
best_itemKNN = implicit.nearest_neighbours.TFIDFRecommender(K=312)
best_itemKNN.fit(user_items)  # fit на item_users

# рекомендации для тестовых пользователей
item_recs = recommend_with_model(best_itemKNN, user_items, list(test_items.keys()), topn=10)


# --- item-KNN (лучшие параметры) ---
best_userKNN = PET_UserKNN_with_Implicit(K=991, similarity="bm25")
best_userKNN.fit(train_matrix)

# рекомендации для тестовых пользователей
user_recs = recommend_with_model(best_userKNN, train_matrix, list(test_items.keys()), topn=10)

  0%|          | 0/3662 [00:00<?, ?it/s]

  0%|          | 0/5392 [00:00<?, ?it/s]

### Оцениваем

In [ ]:
item_KNN_results = model_evaluation(item_recs,test_items, all_items, 10, 'itemKNN_tfidf_k=312')
item_KNN_results

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
itemKNN_tfidf_k=312,0.812201,0.329067,0.040669,0.345263,0.233586,0.074823


In [ ]:
user_KNN_results = model_evaluation(user_recs,test_items, all_items, 10, 'userKNN_bmp25_k=991')
user_KNN_results

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
userKNN_bmp25_k=991,0.836124,0.341029,0.043678,0.361278,0.244691,0.0639


### Сохраняем результаты эксперимента

In [ ]:
results_data, json_file, csv_file = save_experiment_results(
    result=item_KNN_results,
    model_name="itemKNN_tfidf_k=312",
    meta=meta,
    results_dir = RESULTS_DIR
)

Результат добавлен в существующий CSV файл
JSON результат сохранен как: itemKNN_tfidf_k=312_20251108_084840.json
CSV со всеми экспериментами: all_experiments_results.csv
Все результаты в: /content/drive/MyDrive/Colab Notebooks/data/results

СВОДКА ЭКСПЕРИМЕНТА
Модель: itemKNN_tfidf_k=312
Метка времени: 20251108_084840
Дата оценки: 2025-11-08T08:48:40
Размер train: 800,142
Размер test: 94,842
Пользователей в test: 836
Уникальных предметов: 3662
HitRate@10: 81.2%
precision@10: 32.91%
recall@10: 4.07%
ndcg@10: 34.53%
map@10: 23.36%
Coverage@10: 7.48%

Последние эксперименты (2 всего):


,model_name,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10,timestamp,evaluation_date
0,Most_Popular,0.515550,0.132895,0.015591,0.133171,0.068656,0.002731,20251022_194247,2025-10-22T19:42:47.576490
1,itemKNN_tfidf_k=312,0.812201,0.329067,0.040669,0.345263,0.233586,0.074823,20251108_084840,2025-11-08T08:48:40.783074


In [ ]:
results_data, json_file, csv_file = save_experiment_results(
    result=user_KNN_results,
    model_name="userKNN_bmp25_k=991",
    meta=meta,
    results_dir = RESULTS_DIR
)

Результат добавлен в существующий CSV файл
JSON результат сохранен как: userKNN_bmp25_k=991_20251108_084943.json
CSV со всеми экспериментами: all_experiments_results.csv
Все результаты в: /content/drive/MyDrive/Colab Notebooks/data/results

СВОДКА ЭКСПЕРИМЕНТА
Модель: userKNN_bmp25_k=991
Метка времени: 20251108_084943
Дата оценки: 2025-11-08T08:49:43
Размер train: 800,142
Размер test: 94,842
Пользователей в test: 836
Уникальных предметов: 3662
HitRate@10: 83.6%
precision@10: 34.10%
recall@10: 4.37%
ndcg@10: 36.13%
map@10: 24.47%
Coverage@10: 6.39%

Последние эксперименты (3 всего):


,model_name,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10,timestamp,evaluation_date
0,Most_Popular,0.515550,0.132895,0.015591,0.133171,0.068656,0.002731,20251022_194247,2025-10-22T19:42:47.576490
1,itemKNN_tfidf_k=312,0.812201,0.329067,0.040669,0.345263,0.233586,0.074823,20251108_084840,2025-11-08T08:48:40.783074
2,userKNN_bmp25_k=991,0.836124,0.341029,0.043678,0.361278,0.244691,0.063900,20251108_084943,2025-11-08T08:49:43.731391


### Чтение результата

In [ ]:
csv_path = RESULTS_DIR/"all_experiments_results.csv"
results_df = pd.read_csv(csv_path)
results_df

,model_name,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10,timestamp,evaluation_date
0,Most_Popular,0.515550,0.132895,0.015591,0.133171,0.068656,0.002731,20251022_194247,2025-10-22T19:42:47.576490
1,itemKNN_tfidf_k=312,0.812201,0.329067,0.040669,0.345263,0.233586,0.074823,20251108_084840,2025-11-08T08:48:40.783074
2,userKNN_bmp25_k=991,0.836124,0.341029,0.043678,0.361278,0.244691,0.063900,20251108_084943,2025-11-08T08:49:43.731391


# **Итого:**



## **Наблюдения:**
**User-KNN (bmp25, k=991)**  
- чемпион! лучшие  : hit_rate, precision, recall, ndcg, map.
- Достаточно высокое покрытие (≈6.4%).

**Most Popular**
- Простая модель, но даёт не нулевой baseline: precision@10 = 0.133
- сильно проигрывает userKNN по всем метрикам.(  

**Item-KNN (tfidf, k=312)**
- лучшее покрытие (≈7.5%), то есть рекомендации более разнообразные, но немного отстаёт от кастомного userKNN

---

## **Выводы**
- User-KNN - сильный baseline, который стоит взять за основу.

- Item-KNN - чуть слабее по качеству, но лучше по coverage — может быть полезен для увеличения новизны рекомендаций.

- Most Popular - полезен как контрольная точка, или cold_start(для новых пользователей).


# **P.S.**
- Вопрос: почему такой низкий Recall?
- Ответ: в test есть ползователи у которых просмотрены не 10 фильмов а более, соответственно масксимальный Recall < 1. Представим , что наша модель идеальна и взглянем на максимально_достижимый Recall при top_10.

In [20]:
top_n = 10
max_recall = (top_n /test.groupby('user_id')['item_id'].count()).mean()
print (f"Максимально достижимый recal = {max_recall:.4f}")

Максимально достижимый recal = 0.2481


# '######################################################'